In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from pprint import pprint
import os.path as pth

import gzip
import pickle

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Flatten, Reshape
from tensorflow.keras.utils import Sequence

from sklearn.model_selection import train_test_split, KFold, GroupKFold

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
ROOT_DIR = '../data'
TRAIN_IMG_DIR = pth.join(ROOT_DIR, 'original/train_images')
TRAIN_DATA_DIR = ROOT_DIR

WIDTH=1600
HEIGHT=256

In [ ]:
data = pd.read_feather(pth.join(TRAIN_DATA_DIR, 'train.feather'))

In [ ]:
image_groups = data.groupby('imageid')

s = image_groups[['mask_present']].sum().mask_present
s.name = 'nmasks'
data = data.merge(s, left_on=['imageid'], right_index=True)

In [ ]:
data.head()

In [ ]:
with gzip.open(pth.join(TRAIN_DATA_DIR, 'train_masks.pickle.gz'), 'rb') as f:
    image_masks = pickle.load(f)

from loaders import BlockwiseImageLoader
n = 4
b = BlockwiseImageLoader(train_cv_image_names[0:10], image_groups, image_masks, n, 128, 64)
img_patches, mask_patches = b[0]
imgs = b.combine_patches(img_patches)

for i in range(n):
    name = train_cv_image_names[i]
    im = plt.imread(pth.join(TRAIN_IMG_DIR, f'{name}.jpg')).astype('float32')[:, :, 0]
    plt.figure()
    plt.imshow(im, cmap='gray')
    plt.show()
    plt.figure()
    plt.imshow(imgs[i], cmap='gray')
    plt.show()


In [254]:
from models import SimpleAE, SimpleInception, UNet

model = UNet()
model.build_model()
model.summary()

2 Tensor("max_pooling2d_221/MaxPool:0", shape=(?, 128, 800, 64), dtype=float32) Tensor("conv2d_1227/Relu:0", shape=(?, 256, 1600, 64), dtype=float32)
2 Tensor("max_pooling2d_222/MaxPool:0", shape=(?, 64, 400, 128), dtype=float32) Tensor("conv2d_1229/Relu:0", shape=(?, 128, 800, 128), dtype=float32)
2 Tensor("max_pooling2d_223/MaxPool:0", shape=(?, 32, 200, 256), dtype=float32) Tensor("conv2d_1231/Relu:0", shape=(?, 64, 400, 256), dtype=float32)
2 Tensor("max_pooling2d_224/MaxPool:0", shape=(?, 16, 100, 512), dtype=float32) Tensor("conv2d_1233/Relu:0", shape=(?, 32, 200, 512), dtype=float32)
1 Tensor("conv2d_1235/Relu:0", shape=(?, 16, 100, 1024), dtype=float32)
Tensor("conv2d_1235/Relu:0", shape=(?, 16, 100, 1024), dtype=float32)
Model: "model_54"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_74 (InputLayer)           [(None, 256,

In [ ]:
image_names = list(data[data.nmasks > 0].imageid.unique())[0:1000]
train_cv_image_names, test_image_names = train_test_split(image_names)
len(train_cv_image_names), len(test_image_names)

In [ ]:
from loaders import ImageLoader, BlockwiseImageLoader

BATCH_SIZE = 4
PATCH_SIZE = 64
PATCH_STRIDE = 32

kfold = KFold(2)

for train_indices, cv_indices in kfold.split(train_cv_image_names):
    train_image_names = [train_cv_image_names[i] for i in train_indices]
    cv_image_names = [train_cv_image_names[i] for i in cv_indices]
    print(f'{len(train_image_names)} training samples, {len(cv_image_names)} validation samples')
    model.fit_generator(
        generator=ImageLoader(train_image_names, image_groups, image_masks, 
                              BATCH_SIZE),#, PATCH_SIZE, PATCH_STRIDE),
        validation_data=ImageLoader(cv_image_names, image_groups, image_masks, 
                                    BATCH_SIZE),#, PATCH_SIZE, PATCH_STRIDE),
        epochs=5)

In [ ]:
model.model.history.history.keys()

In [ ]:
def plot_history(history, keys=['loss', 'val_loss']):
    if len(keys) == 0:
        return
    
    n = 3
    primary = np.array(history[keys[0]])
    ymin = primary.mean() - primary.std() * n
    ymax = primary.mean() + primary.std() * n
    
    plt.figure()
    plt.ylim(ymin, ymax)
    for key in keys:
        if key in history.keys():
            #plt.plot(np.arange(len(history[key])), history[key], label=key)
            plt.plot(history[key], label=key)
        else:
            print(f'Unable to plot {key}')
    plt.legend()
    plt.show()
    
plot_history(model.model.history.history)
    

In [ ]:
names = test_image_names[0:10]
loader = ImageLoader(names, image_groups, image_masks,
                     BATCH_SIZE)#, PATCH_SIZE, PATCH_STRIDE)
results = model.predict_generator(generator=loader)
#results = loader.combine_mask_patches(results)

In [ ]:
for i, name in enumerate(names):
    im = plt.imread(pth.join(TRAIN_IMG_DIR, f'{name}.jpg')).astype('float32')
    im /= 255
    
    print(name)
    plt.figure(figsize=(16, 3))
    plt.imshow(im, cmap='gray')
    plt.show()
    
    for j in [1, 2, 3, 4]:
        rows = data[(data.imageid == name) & (data.classid == j)]
        row = rows.iloc[0, :]

        if row.mask_present:
            mask_ref = image_masks[f'{name}_{j}']
        else:
            mask_ref = np.zeros((HEIGHT, WIDTH), dtype='uint8')
            
        print(name, j, 'REF')
        plt.figure(figsize=(16, 3))
        plt.imshow(mask_ref, cmap='gray')
        plt.show()
        
        mask_test = results[j - 1][i].squeeze()
        #mask_test = (mask_test > 0.5).astype('uint8')
        
        
        print(name, j, 'TEST', np.sum(mask_ref != mask_test))
        plt.figure(figsize=(16, 3))
        plt.imshow(mask_test, cmap='gray')
        plt.show()
